<a href="https://colab.research.google.com/github/chocozzang/My_Python_Practice/blob/main/%EC%9E%91%EC%97%85_2%EC%9C%A0%ED%98%95_%ED%88%AC%ED%91%9C%EA%B8%B0%EB%B0%98%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/breast-cancer-wisconsin.csv', encoding='utf-8')

data = data[data['bare_nucleoli'].str.isnumeric()]

x = data[data.columns[1:10]]
y = data[['class']]

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
x_scaled_train = scaler.transform(x_train)
x_scaled_test  = scaler.transform(x_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

logit_model = LogisticRegression(random_state = 42)
rnf_model   = RandomForestClassifier(random_state = 42)
svc_model   = SVC(random_state = 42)

voting_hard = VotingClassifier(estimators = [('lr', logit_model), ('rf', rnf_model), ('svc', svc_model)], voting = 'hard')
voting_hard.fit(x_scaled_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [18]:
from sklearn.metrics import accuracy_score

for clf in (logit_model, rnf_model, svc_model, voting_hard) :
  clf.fit(x_scaled_train, y_train)
  y_pred = clf.predict(x_scaled_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544
VotingClassifier 0.9649122807017544


In [22]:
from sklearn.metrics import confusion_matrix

logit_pred_train = logit_model.predict(x_scaled_train)
logit_pred_test  = logit_model.predict(x_scaled_test)
#
#svc_train        = svc_model.predict(x_scaled_train)
logit_cm_train = confusion_matrix(y_train, logit_pred_train)
print("logistic train model confusion matrix : \n", logit_cm_train)
logit_cm_test = confusion_matrix(y_test, logit_pred_test)
print("logistic test model confusion matrix : \n", logit_cm_test)

logistic train model confusion matrix : 
 [[328   5]
 [  9 170]]
logistic test model confusion matrix : 
 [[106   5]
 [  2  58]]


In [23]:
from sklearn.metrics import confusion_matrix

rnf_pred_train = rnf_model.predict(x_scaled_train)
rnf_pred_test  = rnf_model.predict(x_scaled_test)

rnf_cm_train = confusion_matrix(y_train, rnf_pred_train)
print("logistic train model confusion matrix : \n", rnf_cm_train)
rnf_cm_test = confusion_matrix(y_test, rnf_pred_test)
print("logistic test model confusion matrix : \n", rnf_cm_test)

logistic train model confusion matrix : 
 [[333   0]
 [  0 179]]
logistic test model confusion matrix : 
 [[106   5]
 [  1  59]]


In [25]:
from sklearn.metrics import confusion_matrix

svc_pred_train = svc_model.predict(x_scaled_train)
svc_pred_test  = svc_model.predict(x_scaled_test)

svc_cm_train = confusion_matrix(svc_pred_train, y_train)
print("svc train model confusion matrix : \n", svc_cm_train)
svc_cm_test  = confusion_matrix(svc_pred_test,  y_test)
print("svc test model confusion matrix : \n", svc_cm_test)

svc train model confusion matrix : 
 [[329   4]
 [  4 175]]
svc test model confusion matrix : 
 [[106   1]
 [  5  59]]


In [26]:
from sklearn.metrics import confusion_matrix

voting_pred_train = voting_hard.predict(x_scaled_train)
voting_pred_test  = voting_hard.predict(x_scaled_test)

voting_cm_train = confusion_matrix(y_train, voting_pred_train)
print("voting train model confustion matrix : \n", voting_cm_train)
voting_cm_test  = confusion_matrix(y_test, voting_pred_test)
print("voting test model confusion matrix : \n", voting_cm_test)

voting train model confustion matrix : 
 [[329   4]
 [  4 175]]
voting test model confusion matrix : 
 [[106   5]
 [  1  59]]


In [30]:
svm_model   = SVC(probability = True, random_state = 42)
voting_soft = VotingClassifier(estimators = [('lr', logit_model), ('rf', rnf_model), ('svc', svm_model)], voting = 'soft')
voting_soft.fit(x_scaled_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [31]:
from sklearn.metrics import accuracy_score

for clf in (logit_model, rnf_model, svm_model, voting_soft) :
  clf.fit(x_scaled_train, y_train)
  y_pred = clf.predict(x_scaled_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544
VotingClassifier 0.9649122807017544


In [33]:
voting_pred_train2 = voting_soft.predict(x_scaled_train)
voting_train_confusion_matrix2  = confusion_matrix(y_train, voting_pred_train2)

voting_pred_test2 = voting_soft.predict(x_scaled_test)
voting_test_confusion_matrix2 = confusion_matrix(y_test, voting_pred_test2)

print("voting soft train confusion matrix : \n", voting_train_confusion_matrix2 )
print("voting soft test confusion matrix : \n", voting_test_confusion_matrix2 )

voting soft train confusion matrix : 
 [[330   3]
 [  3 176]]
voting soft test confusion matrix : 
 [[106   5]
 [  1  59]]


In [34]:
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/house_price.csv', encoding='utf-8')

x2 = data2[data2.columns[1:5]]
y2 = data2[['house_value']]

In [36]:
from sklearn.model_selection import train_test_split

x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, random_state = 42)

In [37]:
from sklearn.preprocessing import MinMaxScaler

scaler2 = MinMaxScaler()
scaler2.fit(x_train2)
x_scaled_train2 = scaler2.transform(x_train2)
x_scaled_test2  = scaler2.transform(x_test2)

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

linear_model = LinearRegression()
rnf_model    = RandomForestRegressor(random_state = 42)

voting_regressor = VotingRegressor(estimators = [('lr', linear_model), ('rf', rnf_model)])
voting_regressor.fit(x_scaled_train2, y_train2)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(random_state=42))])

In [43]:
pred_train2 = voting_regressor.predict(x_scaled_train2)
voting_regressor.score(x_scaled_train2, y_train2)

0.7962532705428835

In [44]:
pred_test2 = voting_regressor.predict(x_scaled_test2)
voting_regressor.score(x_scaled_test2, y_test2)

0.5936371957936409

In [45]:
import numpy as np
from sklearn.metrics import mean_squared_error

MSE_train = mean_squared_error(y_train2, pred_train2)
MSE_test = mean_squared_error(y_test2, pred_test2)

print("RMSE Train : ", np.sqrt(MSE_train))
print("RMSE Test  : ", np.sqrt(MSE_test))

RMSE Train :  43082.050654857834
RMSE Test  :  60942.38524353489
